This notebook was done by Felix Hagenbrock (it contains some duplicate code with the neural network notebooks)

**Loading images and labels**

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(data_dir, augment=False):
    images, labels = [], []
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

    for label in os.listdir(data_dir):
        for image_file in os.listdir(os.path.join(data_dir, label)):
            img = image.load_img(os.path.join(data_dir, label, image_file), target_size=(256, 256))
            img_tensor = image.img_to_array(img)
            img_tensor = np.expand_dims(img_tensor, axis=0)
            img_tensor /= 255.  # normalize to [0,1] range

            if augment:
                aug_images = [img_tensor]
                for batch in datagen.flow(img_tensor, batch_size=1):
                    aug_images.append(batch)
                    if len(aug_images) >= 6:  # original + 5 augmented images
                        break
                img_tensor = np.concatenate(aug_images, axis=0)

            images.append(img_tensor)
            labels.extend([label] * len(img_tensor))

    return np.concatenate(images, axis=0), np.array(labels)

images, labels = load_data('ds', augment=True)

**DS split**

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
X_train.shape

(3216, 256, 256, 3)

**Model teaching**

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import tensorflow as tf


tf.random.set_seed(42)

model = Sequential()
model.add(Flatten(input_shape=(256, 256, 3)))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax')) 

#tf.keras.utils.plot_model(model)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))


c:\DevTools\Python\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
101/101 ━━━━━━━━━━━━━━━━━━━━ 56s 543ms/step - accuracy: 0.4599 - loss: 4.7428 - val_accuracy: 0.5647 - val_loss: 1.0710
Epoch 2/3
101/101 ━━━━━━━━━━━━━━━━━━━━ 54s 536ms/step - accuracy: 0.5470 - loss: 1.0644 - val_accuracy: 0.5647 - val_loss: 1.0426
Epoch 3/3
101/101 ━━━━━━━━━━━━━━━━━━━━ 54s 532ms/step - accuracy: 0.5470 - loss: 1.0385 - val_accuracy: 0.5647 - val_loss: 1.0207


**Predict and evaluate**

In [4]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred)
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))

print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred, target_names=le.classes_))



26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
{0: 996, 1: 1790, 2: 430}
{0: 228, 1: 454, 2: 122}
Accuracy: 0.5646766169154229
              precision    recall  f1-score   support

        Fist       0.00      0.00      0.00       228
        Palm       0.56      1.00      0.72       454
       Thumb       0.00      0.00      0.00       122

    accuracy                           0.56       804
   macro avg       0.19      0.33      0.24       804
weighted avg       0.32      0.56      0.41       804



c:\DevTools\Python\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\DevTools\Python\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\DevTools\Python\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Solution for first rough pipeline:

Accuracy: 57%

Problem: Dataset is very imbalanced because category palm has 374 images, fist has 200 and thumb has 92. So neural network just pivots toward classificating everything as palm.